## Loading and Preprocessing the Pictures

In [ ]:
import numpy as np
import pandas as pd
import os

from PIL import Image

In [ ]:
def load_images(images_paths, labels_paths, valid_labels):
    images = []
    labels = []

    no_label_counter = 0
    unidentified_counter = 0

    for images_path, labels_path in zip(images_paths, labels_paths):

        # Load labels from csv file
        labels_df = pd.read_csv(labels_path, sep=';')
        labels_df['Filename'] = [file[:-3] + 'png' for file in labels_df['Filename']]

        for filename in os.listdir(images_path):
            # Load the image using PIL
            img_path = os.path.join(images_path, filename)
            img = Image.open(img_path)

            # Crop the image to the size of the spectrogram
            left, upper, right, lower = 55, 36, 389, 252
            img = img.crop((left, upper, right, lower))

            # Convert image to numpy array and normalize
            img_array = np.array(img)[:, :, :3] / 255.0

            # Extract class label from the CSV file based on the image filename
            label_row = labels_df.loc[labels_df['Filename'] == filename]
            
            if label_row.empty:
                no_label_counter += 1
                continue

            label = label_row['Species'].values[0]

            if label not in valid_labels:
                unidentified_counter += 1
                continue
                
            labels.append(label)
            images.append(img_array)
                

    if no_label_counter:
        print(f'Label not found for {no_label_counter} images : Images will not be used.')
    if unidentified_counter:
        print(f'Bat unidentified for {unidentified_counter} images : Images will not be used.')

    return np.array(images), np.array(labels)

In [ ]:
images_folders = ['./Data/dataset1', './Data/dataset2', './Data/dataset3']
labels_paths = ['./Data/dataset1_classified.csv', './Data/dataset2_classified.csv', './Data/dataset3_classified.csv']

valid_labels = ['Bartfledermaus', 'Bechsteinfledermaus', 'Fransenfledermaus',
                'Große Hufeisennase', 'Hufeisennase', 'Mausohr',
                'Langohrfledermaus', 'Wasserfledermaus', 'Wimperfledermaus']

images, labels = load_images(images_folders, labels_paths, valid_labels)
print('Images shape: ', images.shape)
print('Labels shape: ', labels.shape)

In [ ]:
#TODO Data Augmentation ?

## Data Exploration

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [ ]:
num_samples = 12
indices_to_visualize = random.sample(range(len(images)), num_samples)

num_cols = 4
num_rows = (num_samples + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))

for i, index in enumerate(indices_to_visualize):
    row = i // num_cols
    col = i % num_cols

    axes[row, col].imshow(images[index])
    axes[row, col].set_title(f'Class: {labels[index]}')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
sns.countplot(x=labels)
plt.title('Class Distribution')
plt.xticks(rotation=45, ha='right')
plt.show()

label_counts = pd.Series(labels).value_counts()
print(label_counts)

In [ ]:
print('Mean of images: ', np.mean(images))
print('Std deviation of images: ', np.std(images))

empty_string_indices = labels == ''
print('Missing values in labels: ', np.sum(empty_string_indices))
print('Missing values in images: ', np.isnan(images).sum())

## Setting up a CNN Model

In [ ]:
import tensorflow as tf
from keras import layers, models

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Converting to numerical labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(encoded_labels),
                                                  y=encoded_labels)

# Convert class weights to a dictionary
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
# Define a function to build the model with hyperparameters
def build_model(hp):
    model = models.Sequential()

    model.add(layers.Conv2D(hp.Int('conv1_units', min_value=16, max_value=64, step=16), (3, 3), activation='relu', input_shape=(216, 334, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv3_units', min_value=64, max_value=256, step=64), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=64, max_value=512, step=64), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Dense(len(label_encoder.classes_), activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')), 
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Instantiate the tuner
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=20, directory='./cnn-model/tuner_results', project_name='cnn_tuner',
                     overwrite=True)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define model checkpoint callback
checkpoint_path = './cnn-model/model_checkpoint.keras'
model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max')

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=25, validation_split=0.2, class_weight=class_weights_dict, callbacks=[early_stopping, model_checkpoint])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Save the best model
best_model.save('./cnn-model/best_model.keras')

In [ ]:
best_model.summary()

In [ ]:
# Load the best model
cnn_model = tf.keras.models.load_model('./cnn-model/best_model.keras')

In [ ]:
# Train the best model again
history = cnn_model.fit(X_train, y_train, epochs=50, class_weight=class_weights_dict, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Save the trained model
cnn_model.save('./cnn-model/final_model.keras')

In [ ]:
# Load the trained model
cnn_final_model = tf.keras.models.load_model('./cnn-model/final_model.keras')

## Analyzing Results

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Evaluate the best model on the test set
test_loss, test_acc = cnn_final_model.evaluate(X_test, y_test)

In [ ]:
all_classes = label_encoder.classes_

y_pred_probabilities = cnn_final_model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)
y_test_int = np.argmax(tf.keras.utils.to_categorical(y_test), axis=1)

# Extract unique classes present in the test dataset
unique_classes_in_test = np.unique(y_test_int)

# Filter the all_classes list to include only those present in the test dataset
target_classes = [all_classes[i] for i in unique_classes_in_test]

# Convert the integer labels to string labels using the label_encoder
y_test_int_str = label_encoder.inverse_transform(y_test_int)
y_pred_str = label_encoder.inverse_transform(y_pred)

In [ ]:
# Generate confusion matrix
confusion_m = confusion_matrix(y_test_int, y_pred)

plt.figure(figsize=(6, 6))
sns.heatmap(confusion_m, annot=True, fmt='d', cmap='Greens', xticklabels=target_classes, yticklabels=target_classes)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# Classification report
print(classification_report(y_test_int_str, y_pred_str, target_names=target_classes, zero_division=1))